<a href="https://colab.research.google.com/github/emitejadaa/NeuroLinked/blob/main/notebooks/model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne gin-cli braindecode moabb --quiet

from google.colab import drive
import os

drive.mount('/content/drive')
DATA_DIR = "/content/drive/MyDrive/Colab-Dataset/HGD_data"

os.makedirs(DATA_DIR, exist_ok=True)

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.9/282.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.3/242.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.8/248.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.8/166.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 21.7 MB/s eta 0:00:00

In [ ]:
if not os.path.exists(os.path.join(DATA_DIR, "data")):
    !gin get robintibor/high-gamma-dataset
    %cd high-gamma-dataset
    !gin download --content

    !mv data "$DATA_DIR/"
    %cd /content
else:
    print("dataset ya disponible en Drive.")

In [ ]:
import mne
import numpy as np
from glob import glob

# Ejemplo: cargar todos los .edf de entrenamiento del sujeto 1
edf_files = glob(os.path.join(DATA_DIR, "data", "train", "*.edf"))
print(f"Archivos EDF encontrados: {len(edf_files)}")

# Lista para datos y etiquetas
X_raw = []
y_labels = []

for file_path in edf_files:
    raw = mne.io.read_raw_edf(file_path, preload=True, verbose=False)

    # Extraer anotaciones (eventos/clases)
    events, event_id = mne.events_from_annotations(raw, verbose=False)

    X_raw.append(raw)
    y_labels.extend(events[:, -1])  # Última columna = ID de clase

print(f"Registros cargados: {len(X_raw)}")
print(f"Clases detectadas: {set(y_labels)}")


In [ ]:
for raw in X_raw:
    raw.filter(1., 40., fir_design='firwin', verbose=False)

In [ ]:
epochs_all = []
labels_all = []

for raw in X_raw:
    events, event_id = mne.events_from_annotations(raw, verbose=False)
    epochs = mne.Epochs(raw, events, event_id=event_id, tmin=0, tmax=4,
                        baseline=None, preload=True, verbose=False)

    epochs_all.append(epochs.get_data())   # (n_trials, n_channels, n_times)
    labels_all.extend(epochs.events[:, -1])

# Concatenar todos
X = np.vstack(epochs_all)
y = np.array(labels_all)

print(f"Forma final de X: {X.shape}")
print(f"Forma final de y: {y.shape}")

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")